# Семинар 7. Детекция

![](images/1.jpeg)

*Пример детекции*

Детекция - это такая задача, в которой нейронная сеть (детектор) принимает картинку. В результате детекции получается список из задетектированных объектов (детектов) вида

$$[(b_x, b_y, b_h, b_w, p_{obj}, p_{1}, ..., p_{k}), ...]$$

т.е. координат объекта (бокса), вероятности нахождения объекта в боксе и вероятности принадлежать каждому из $k$ классов

Почему мы вообще можем такое делать при помощи свёрточной нейросети?

[Andrew Ng об sliding window](https://www.youtube.com/watch?v=5e5pjeojznk)

Детекторы бывают

- одностадийные (one-stage)
- двустадийные (two-stage)

Двустадийные детекторы исторически появились раньше, поэтому начнём обсуждение с них

# Two-Stage Detectors

## RCNN

1. Берётся предварительно обученная CNN (например, ResNet на ImageNet'е)

2. По исходному изборажению определяются regions proposals - области, в которых могут содержаться объекты - при помощи алгоритма под названием [selective search](https://www.geeksforgeeks.org/selective-search-for-object-detection-r-cnn/). Selective search делит изображение на сегменты, используя информацию о цвете соседних пикселей и т.п., а потом иерархически их объединяет. Так как нам нужны разные масштабы, получается много боксов

![](https://production-media.paperswithcode.com/methods/Screen_Shot_2020-06-08_at_11.44.51_AM_cltn2Mh.png)

3. Боксы отдаются на вход свёрточной сети. Свёрточная сеть выдаёт фичи (векторы, которые получаются при прохождении картинкой части CNN до fc-слоя). Дальше происходит классификация по фичам при помощи [Support Vector Machine](https://towardsdatascience.com/support-vector-machine-introduction-to-machine-learning-algorithms-934a444fca47), уточняются координаты боксов (регрессия) и выдаётся confidence, что в боксе есть объект

![](https://lilianweng.github.io/posts/2017-12-31-object-recognition-part-3/RCNN.png)

4. NMS (Non-Maximum-Supression) - подавляет лишние детекты (по умолчанию у нас их 2k на каждый region proposal)

Недостатки: медленно
- Прогоняется CNN для каждого region proposal'а
- Генерация 2000 region proposal'ов для каждой картинки при помощи selective search

### NMS

Модель находит несколько боксов для одного и того же объекта. Мы же хотим на выходе сетки один бокс для одного объекта, причем хорошего качества

Алгоритм NMS:

1. Отсортировать все боксы по confidence score и отбросить те, у которых он совсем низкий (ниже confidence threshold)
2. Для оставшихся боксов (в цикле):
  - Выбрать бокс с лучшим confidence score
  - Убрать боксы с IOU больше порога (н., IOU > 0.5) с предыдущим боксом

![](images/4.jpg)

[Andrew Ng объясняет NMS](https://www.youtube.com/watch?v=VAo84c1hQX8)

## Fast-RCNN

1. Region proposals определяются так же, как и раньше, при помощи selective search - по исходному изображению
2. Вся картинка (а не каждый region proposal отдельно) идёт на свёрточную сеть
3. Прогоняем всё изображение через сеть и получаем карту признаков
4. Из карты признаков "вырезаем" тензоры, которые соответствуют region proposal'ам. Для этого используется специальный слой ROI pooling
5. Отправляем дальше эти тензоры на классификацию, регрессию координат (классификация уже при помощи fc-слоя, а не SVM)
5. NMS

Такое простое изменение помогло значительно ускорить детекцию (для примера: 50 секунд → 2 секунды, x25). Теперь самая долгая часть всего этого пайплайна детекции - это selective search

![](images/5.png)

## Faster-RCNN

### Anchor boxes

![](images/6.png)

- Помогают предсказывать разные объекты, которые оказались в одном суперпикселе
- Сети будет проще обучиться подгонять регрессией боксы к нужной форме, если задать ей адекватные anchor box'ы
- Не очень понятно, как нейронную сеть заставить выдавать выходы произвольного (или меняющегося) размера: один бокс для 1й картинки, 5 боксов для 2й. Когда у нас есть anchor box'ы, мы учим сеть находить предопределенное заранее количество боксов, которые потом мы используем или не используем дальше для обучения (в зависимости от того, попал туда объект или нет)

### Архитектура Faster-RCNN

Поскольку теперь самая долгая часть - это selective search, то логичнее всего для ускорения детекции предложить заменить этот алгоритм на более быстрый. Теперь у нас будет отдельная сеть для определения region proposals - Region Proposal Network (RPN). Остальная часть будет как в fast-RCNN. Всё вместе это будет называться Faster-RCNN

![](images/7.png)

Как работает RPN:

1. Скользящее окно движется по карте признаков, общей с fast-RCNN (получается, что region proposals мы ищем уже по карте признаков, а не по исходной картинке)
2. Центр скользящего окна связан с центром якорей (anchor box'ов). Используются anchor box'ы разной формы и размеров. Это как бы "заготовка" для более точного бокса
3. Если IOU anchor box'а с ground truth выше порога (например, 70%), то мы говорим, что это положительный якорь, а если ниже порога (например, 30%) - то это отрицательный якорь. Все остальные якоря не участвуют в обучении

  Положительный якорь → значит, бокс соответствует нашему объекту → эти боксы учим регрессией быть более похожими на ground truth. Также учим их выдавать тот же класс, что и ground truth (кросс-энтропия)

  Отрицательный якорь → значит, бокс точно не соответствует нашему объекту → эти боксы в регрессии не участвуют. Также учим их выдавать противоположный класс на классификации

  **А если вкратце, RPN "отвечает" на два вопроса: содержит ли этот якорь объект и, если да, то как нужно поменять координаты якоря, чтобы он соответствовал найденному объекту. Дальше RCNN модуль классифицирует бокс и уточняет его координаты (если объект найден) или говорит, что это фон.**

### Summary

![](images/8.png)

*Two-stage detectors, изображение из блога [lilianweng.github.io](https://lilianweng.github.io/posts/2017-12-31-object-recognition-part-3/). Справа модель Mask-RCNN, которую мы не обсуждаем в рамках данной темы*

# One-Stage Detectors

- Ранее детектирование производилось путём выбора областей изображения и их классификации
- Модель для детектирования состояла из нескольких частей
- Подобные сети требуют большого времени на предсказание, а ещё больше на обучение сетей

## SSD

- В качестве бэкбона используется `VGG16`, обученная на `ImageNet`
- К бэнбону добавляется серия свёрток
- Детекция происходит на каждой из этих свёрток, чтобы лучше работать с разными масштабами
- Pre-defined anchors

![](images/9.png)

Anchor box'ы для разных масштабов:

![](images/10.png)

## YOLO - You Only Look Once

- Первая версия без anchor box'ов. Изображение делилось на $S\times S$ клеток, в каждой из которых предсказывалось $B$ штук bounding box'ов, степень уверенности на каждый бокс и $C$ штук вероятностей классов на одну ячейку
- В основе лежит `GoogleNet`, в которой `inception`-блоки заменили на conv1x1 + conv3x3

После свёрток получаем карту признаков. В двухстадийном детекторе мы сначала находили regions of interest (ROI), а потом уже их уточняли при помощи регрессии. Раз мы всё равно уточняем координаты бокса, то будем поступать проще: поделим изображение на непересекающиеся патчи и каждый из них будем считать нашим ROI. А потом для каждого патча мы уточним бокс:

![](images/11.jpg)

Плюсы:

- Работает в один заход, быстрая

Минусы:

- Каждая ячейка может предсказывать только один класс
- Плохо справляется с маленькими объектами

![](images/12.png)

### [YOLO](https://arxiv.org/abs/1506.02640) (2015) → [YOLO9000](https://arxiv.org/abs/1612.08242v1) (YOLOv2) (2016) → [YOLOv3](https://arxiv.org/abs/1804.02767v1) (2018) → [YOLOv4](https://arxiv.org/abs/2004.10934v1) (2020) → [YOLOv5](https://github.com/ultralytics/yolov5) (2020)

- От предсказания bounding box'ов перешли к предсказанию поправок к anchor'ам
- Добавили skip connection'ы
- Предсказания производятся для 3 разных скейлов, чтобы лучше детектировать объекты разного размера
- Для каждого скейла три anchor box'а
- Anchor'ы можно подобрать под датасет


### Anchor'ы в YOLO

Ситуация становится сложнее, когда в одном суперпикселе несколько объектов. Используем для каждой ячейки anchor box'ы.

Изображение делится на $S\times S$ клеток, в каждой из которых есть $A$ штук anchor box'ов. Можно выбрать их по датасету (кластеризация + усреднение), можно задать руками.

Во время обучения
- каждый объект относится к определенной ячейке
- выбирается anchor box, имеющий максимальный IOU с боксом из датасета

Выход: $S \times S$ клеток $\times$ $A$ якорей $\times (5 + C)$

![](images/12.jpg)

### Разные скейлы в YOLO

![](images/13.png)

Структура одностадийного детектора. Карты признаков для разных масштабов смешиваются между собой, а после этого происходит непосредственно детекция (вспомним `Feature Pyramid Network`):

![](images/14.png)

Вместо классического `Feature Pyramid Network` используют `Path Aggreagtion Network` (более хитрое смешивание фичей с разных масштабов):

![](images/15.png)

## Loss

  $$𝓛 = 𝓛_{reg} + 𝓛_{obj} + 𝓛_{clas}$$
  - регрессия: уточняются координаты, штрафуется за сдвиг центра бокса относительно ground truth и за уширение/удлинение бокса по сравнению с ground truth
  - object loss: учим сеть выдавать object confidence, близкий к 1, в тех боксах, где есть объект (т.е. отличать объект от шума, от заблюренных пикселей, фона и т.п.)
  - классификация: обычная кросс-энтропия


# mAP (Mean Average Precision) - метрика для детекции

Сложная интегральная метрика, нужно в одном числе учесть и точность боксов, и confidence класса. От того, какие пороги по confidence и по IOU мы поставим, будет зависеть, какие детекты мы получим

Как считаем метрику (при фиксированном `IOU_thr` и `conf_thr`):
- **TP**: лучший бокс с $IOU(pred, gt) > IOU\_thr$
- **FP**: $IOU(pred, gt) < IOU\_thr$ или бокс-дубликат
- **FN**: не было предсказано бокса либо модель предсказала бокс, но ошиблась с классом

  Теперь у нас есть нужные величины, чтобы посчитать `precision` и `recall`

*Вопрос: Как выглядят формулы для Precision и Recall?*

Хотим, чтобы наша метрика учитывала то, как влияет изменение порогов на изменение предсказаний сети

Алгоритм:

0. Выбираем один класс, пункты 1 - 4 проделываем для всех классов отдельно
1. Фиксируем `IOU_thr`, пробегаемся в цикле по `conf_thr` (`linspace`).
2. Для выбранных порогов считаем две цифры: `(precision, recall)`.
3. Строим precision-recall curve.
4. Считаем площадь под кривой
5. Полученная величина называется `Average Precision` и зависит она от IOU-порога: `AP(IOU_thr)`
6. Мы можем усреднить полученные для всех классов `AP(IOU_thr)`, назовем такую метрику `mAP(IOU_thr)`


## Разное

[Видео о детекторах на русском](https://www.youtube.com/watch?v=LFrvzlcWthk)